In [2]:
!pip install selenium

In [4]:
!pip3 install webdriver_manager

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv
# Setup the WebDriver (using Firefox in this example)
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
firefox_options = Options()
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)

# Open the Craigslist furniture listing page
url = 'https://indianapolis.craigslist.org/search/ela?query=tv#search=1~gallery~0~0'
driver.get(url)

def get_product_names_categories_descriptions_conditions_placenames_regions_geo_positions_icbms_postids_and_posted_dates():
    product_info = []
    
    # Wait for the page to load
    time.sleep(5)
    
    # Find all product name elements on the current page
    product_name_elements = driver.find_elements(By.XPATH, "//a[contains(@class, 'cl-app-anchor text-only posting-title')]")
    
    for element in product_name_elements:
        product_name = element.find_element(By.CLASS_NAME, 'label').text
        product_type = 'TV' if 'TV' in product_name.upper() else 'NonTV'
        # Get the link to the product's detail page
        product_link = element.get_attribute('href')
        # Open the product's detail page in a new tab
        driver.execute_script("window.open(arguments[0]);", product_link)
        driver.switch_to.window(driver.window_handles[1])
        # Wait for the detail page to load
        time.sleep(3)
        # Find the description element and extract the text
        description_element = driver.find_element(By.ID, 'postingbody')
        description_text = description_element.text
        
        # Try to find the condition element and extract the text
        try:
            condition_element = driver.find_element(By.XPATH, "//p[@class='attrgroup']/span/b")
            condition_text = condition_element.text
        except:
            condition_text = "Not specified"  # Use a placeholder if the condition is not found

        # Try to find the geo.placename, geo.region, geo.position, ICBM, and post id elements and extract the content
        try:
            placename_element = driver.find_element(By.XPATH, "//meta[@name='geo.placename']")
            placename = placename_element.get_attribute('content')
        except:
            placename = "Not specified"  # Use a placeholder if the placename is not found

        try:
            region_element = driver.find_element(By.XPATH, "//meta[@name='geo.region']")
            region = region_element.get_attribute('content')
        except:
            region = "Not specified"  # Use a placeholder if the region is not found

        try:
            geo_position_element = driver.find_element(By.XPATH, "//meta[@name='geo.position']")
            geo_position = geo_position_element.get_attribute('content')
        except:
            geo_position = "Not specified"  # Use a placeholder if the geo.position is not found

        try:
            icbm_element = driver.find_element(By.XPATH, "//meta[@name='ICBM']")
            icbm = icbm_element.get_attribute('content')
        except:
            icbm = "Not specified"  # Use a placeholder if the ICBM is not found

        # Try to find the post id element and extract the content
        try:
            post_id_element = driver.find_element(By.XPATH, "//p[contains(@class, 'postinginfo') and contains(text(), 'post id:')]")
            post_id_text = post_id_element.text.split(': ')[1]  # Assuming the format is "post id: 1234567890"
        except:
            post_id_text = "Not specified"  # Use a placeholder if the post id is not found

        # Try to find the posted date element and extract the datetime attribute
        try:
            posted_date_element = driver.find_element(By.CLASS_NAME, 'timeago')
            posted_date = posted_date_element.get_attribute('datetime')
        except:
            posted_date = "Not specified"  # Use a placeholder if the posted date is not found
        
        # Append product name, category, product type, description, condition, placename, region, geo position, icbm, post id, and posted date
        product_info.append([product_name, 'Electronics', product_type, description_text, condition_text, placename, region, geo_position, icbm, post_id_text, posted_date])
        
        # Close the detail tab and switch back to the main window
        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        # Wait a bit before scraping the next product to avoid being rate-limited
        time.sleep(1)
    
    return product_info

# Extract product names, category, type, description, condition, placename, region, geo position, icbm, post id, and posted date
product_info = get_product_names_categories_descriptions_conditions_placenames_regions_geo_positions_icbms_postids_and_posted_dates()

# Optional: Write the product names, category, type, description, condition, placename, region, geo position, icbm, post id, and posted date to a CSV file
with open('craigslist_Electronics_product.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Product Name', 'Category', 'Type', 'Description', 'Condition', 'Place Name', 'Region', 'Geo Position', 'ICBM', 'Post ID', 'Posted Date'])  # Write header
    for item in product_info:
        writer.writerow(item)

# Close the browser
driver.quit()


C:\Users\PC\AppData\Local\Temp\ipykernel_9200\3725014110.py:11: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options=firefox_options)
